<a href="https://colab.research.google.com/github/deepiskp-99/DSA_Python_CaseStudy/blob/main/vs_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from flask import Flask, request, jsonify
import joblib
import pandas as pd

app = Flask(__name__)

# Load trained model and feature columns
model = joblib.load('airbnb_rf_model.pkl')
model_columns = joblib.load('model_columns.pkl')

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json(force=True)
    input_df = pd.DataFrame([data])

    # Feature engineering: create the same interaction feature
    if 'Neighbourhood' in input_df.columns and 'Room Type' in input_df.columns:
        input_df['Neighbourhood_Room Type'] = (
            input_df['Neighbourhood'] + '_' + input_df['Room Type']
        )

    # One-hot encoding of all categorical features
    input_encoded = pd.get_dummies(input_df)
    # Add missing columns as zeros
    for col in model_columns:
        if col not in input_encoded.columns:
            input_encoded[col] = 0
    input_encoded = input_encoded[model_columns]

    pred = model.predict(input_encoded)[0]
    return jsonify({'prediction': float(pred)})

if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

# 1. Load Data
df = pd.read_excel(r'C:\Users\LAB-USER-01\Desktop\Deepika\partpdf_1763620814447_airbnb.xlsx')

# 2. Handle missing values
for col in df.select_dtypes(include=np.number).columns:
    df[col].fillna(df[col].median(), inplace=True)
for col in df.select_dtypes(include='object').columns:
    df[col].fillna(df[col].mode()[0], inplace=True)

# (Optional) EDA
print(df.describe())
print(df.isnull().sum())
sns.histplot(df['Price'], bins=50)
plt.title('Price distribution')
plt.savefig("price_distribution.png")

# 4. Feature Engineering
df['Neighbourhood_Room Type'] = df['Neighbourhood '] + '_' + df['Room Type']

# One-hot encoding
df_encoded = pd.get_dummies(
    df,
    columns=['Neighbourhood', 'Property Type', 'Room Type', 'Zipcode', 'Neighbourhood_Room Type'],
    drop_first=True
)
df_encoded = df_encoded.astype({c: int for c in df_encoded.select_dtypes('bool').columns})
# 5. Modeling
X = df_encoded.drop(columns=['Price', 'Name'])  # Don't use Name for modeling
y = df_encoded['Price']

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("Final Model RMSE:", rmse)

# 6. Save model and feature columns
joblib.dump(rf, 'airbnb_rf_model.pkl')
joblib.dump(list(X.columns), 'model_columns.pkl')

In [ ]:
body {
    font-family: Arial, sans-serif;
    background: #f0f0f0;
    margin: 0;
    padding: 0;
}
.container {
    max-width: 500px;
    margin: 50px auto;
    background: #fff;
    padding: 30px;
    border-radius: 10px;
    box-shadow: 0 4px 8px rgba(0,0,0,0.13);
}
h1 {
    text-align: center;
}
.form-group {
    margin-bottom: 15px;
}
label {
    display: block;
    margin-bottom: 5px;
}
input, select {
    width: 100%;
    padding: 8px;
    box-sizing: border-box;
    border-radius: 5px;
    border: 1px solid #bbb;
}
button {
    width: 100%;
    padding: 10px;
    background: #0099ff;
    color: #fff;
    border-radius: 6px;
    border: none;
    font-size: 16px;
    cursor: pointer;
}
button:hover {
    background: #007acc;
}
#result {
    margin-top: 20px;
    padding: 15px;
    background: #f6f9fc;
    border-radius: 5px;
    text-align: center;
}

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Airbnb Price Prediction</title>
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <style>
        body { font-family: Arial, sans-serif; background: #f0f0f0; margin: 0; padding: 0;}
        .container { max-width: 500px; margin: 50px auto; background: #fff; padding: 30px; border-radius: 10px; box-shadow: 0 4px 8px rgba(0,0,0,0.13);}
        h1 { text-align: center; }
        .form-group { margin-bottom: 15px; }
        label { display: block; margin-bottom: 5px; }
        input, select { width: 100%; padding: 8px; box-sizing: border-box; border-radius: 5px; border: 1px solid #bbb;}
        button { width: 100%; padding: 10px; background: #0099ff; color: #fff; border-radius: 6px; border: none; font-size: 16px; }
        #result { margin-top: 20px; padding: 15px; background: #f6f9fc; border-radius: 5px; text-align: center; }
    </style>
</head>
<body>
<div class="container">
    <h1>Airbnb Price Prediction</h1>
    <form id="predictionForm">
        <div class="form-group">
            <label for="Neighbourhood">Neighbourhood</label>
            <input type="text" id="Neighbourhood" name="Neighbourhood" required>
        </div>
        <div class="form-group">
            <label for="PropertyType">Property Type</label>
            <input type="text" id="PropertyType" name="Property Type" required>
        </div>
        <div class="form-group">
            <label for="ReviewScoresBin">Review Scores Rating (bin)</label>
            <input type="number" id="ReviewScoresBin" name="Review Scores Rating (bin)" required>
        </div>
        <div class="form-group">
            <label for="RoomType">Room Type</label>
            <input type="text" id="RoomType" name="Room Type" required>
        </div>
        <div class="form-group">
            <label for="Zipcode">Zipcode</label>
            <input type="text" id="Zipcode" name="Zipcode" required>
        </div>
        <div class="form-group">
            <label for="Beds">Beds</label>
            <input type="number" id="Beds" name="Beds" required>
        </div>
        <div class="form-group">
            <label for="Records">Number of Records</label>
            <input type="number" id="Records" name="Number of Records" required>
        </div>
        <div class="form-group">
            <label for="Reviews">Number Of Reviews</label>
            <input type="number" id="Reviews" name="Number Of Reviews" required>
        </div>
        <div class="form-group">
            <label for="ReviewScores">Review Scores Rating</label>
            <input type="number" id="ReviewScores" name="Review Scores Rating" required>
        </div>
        <button type="submit">Predict Price</button>
    </form>
    <div id="result"></div>
</div>
<script>
document.getElementById('predictionForm').addEventListener('submit', function(e){
    e.preventDefault();
    let form = e.target;
    let data = {
        "Neighbourhood": form["Neighbourhood"].value,
        "Property Type": form["Property Type"].value,
        "Review Scores Rating (bin)": parseInt(form["Review Scores Rating (bin)"].value),
        "Room Type": form["Room Type"].value,
        "Zipcode": form["Zipcode"].value,
        "Beds": parseInt(form["Beds"].value),
        "Number of Records": parseInt(form["Number of Records"].value),
        "Number Of Reviews": parseInt(form["Number Of Reviews"].value),
        "Review Scores Rating": parseInt(form["Review Scores Rating"].value)
    };
    fetch('/predict', {
        method: 'POST',
        headers: {'Content-Type': 'application/json'},
        body: JSON.stringify(data)
    })
    .then(res => res.json())
    .then(result => {
        document.getElementById('result').innerHTML = `<strong>Predicted Price:</strong> $${result.prediction.toFixed(2)}`;
    })
    .catch(err => {
        document.getElementById('result').innerHTML = "<span style='color:red'>Prediction failed. Check server.</span>";
    });
});
</script>
</body>
</html>